In this notebook we will curate a math dataset avaliable on huggingface: https://huggingface.co/datasets/ajibawa-2023/Maths-College

In [3]:
# Save the dataset to a local file
import datasets as ds

# Load the dataset
data = ds.load_dataset("ajibawa-2023/Maths-College")
data.save_to_disk("MathDataset")

Generating train split:   0%|          | 0/969980 [00:00<?, ? examples/s]

Saving the dataset (0/10 shards):   0%|          | 0/969980 [00:00<?, ? examples/s]

Now we will load the dataset and make some small changes to the instructions

In [38]:
import datasets as ds
dts = ds.load_from_disk("MathDataset")
def edit_instruction(example):
    old_instruction = "Write an educational piece suited for college students related to the following text snippet:"
    new_instruction = "Write an educational piece related to the following text snippet:"
    example['context'] = example['instruction'].replace(old_instruction, '')
    # remove starting \n from the context if it exists
    if example['context'].startswith('\n'):
        example['context'] = example['context'][1:]
    example['instruction'] = new_instruction
    return example
new_dts = dts.map(edit_instruction)
new_dts.save_to_disk("MathInstructDataset")

Map:   0%|          | 0/969980 [00:00<?, ? examples/s]

Saving the dataset (0/10 shards):   0%|          | 0/969980 [00:00<?, ? examples/s]

Sweet, now we have a dataset that is suted for instruct model tuning. We will use the `transformers` library to fine-tune a model on this dataset.

This edited dataset is now on huggingface [Math-instruct-dataset](https://huggingface.co/datasets/patrickjmcbride/math-instruct-dataset)


In [45]:
# Load the dataset
data = ds.load_dataset("patrickjmcbride/math-instruct-dataset")
data.save_to_disk("MathInstructDataset")

Generating train split:   0%|          | 0/969980 [00:00<?, ? examples/s]

Saving the dataset (0/10 shards):   0%|          | 0/969980 [00:00<?, ? examples/s]

In [46]:
mid = ds.load_from_disk("MathInstructDataset")
print(mid['train'][0]['instruction'])
print("===============================")
print(mid['train'][0]['context'])
print("===============================")
print(mid['train'][0]['output'])

Write an educational piece related to the following text snippet:
"# logical implication

## 1 Short version

Logical implication is an operation on two logical values, typically the values of two propositions (http://planetmath.org/PropositionalCalculus), that produces a value of false just in case the first operand is true and the second operand is false.

The truth table for the logical implication operation that is written as $p\Rightarrow q$ and read as $p\ \operatorname{implies}\ q",$ also written as $p\rightarrow q$ and read as $\operatorname{if}\ p\ \operatorname{then}\ q",$ is as follows:

 Logical Implication $p$ $q$ $p\Rightarrow q$ F F T F T T T F F T T T

## 2 Long version

The mathematical objects that inform our capacity for logical reasoning are easier to describe in a straightforward way than it is to reconcile their traditional accounts. Still, some discussion of the language that occurs in the literature cannot be avoided.

The concept of logical implication encompas

This is all for this notebook, head to finetune.ipynb to continue the process